# Transformers, what can they do?

Course Website: https://huggingface.co/learn/nlp-course/chapter1/3


Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

Also, log into Hugging face

In [2]:
from huggingface_hub import notebook_login

notebook_login()

### Transformers are everywhere!

<font color='blue'>Transformer models</font> are used to solve <font color='blue'>all kinds of NLP tasks</font>, like the ones mentioned in the previous section. Here are some of the companies and organizations using Hugging Face and Transformer models, who also contribute back to the community by sharing their models:

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/companies.PNG)

The [🤗 Transformers library](https://github.com/huggingface/transformers) provides the functionality to <font color='blue'>create</font> and <font color='blue'>use</font> those <font color='blue'>shared models</font>. The [Model Hub](https://huggingface.co/models) contains <font color='blue'>thousands</font> of <font color='blue'>pretrained models</font> that anyone can <font color='blue'>download</font> and <font color='blue'>use</font>. You can also upload your own models to the Hub!

Before diving into how Transformer models work under the hood, let's look at a few examples of how they can be used to solve some interesting NLP problems.

###  Working with pipelines

The most basic object in the 🤗 Transformers library is the <font color='blue'>pipeline() function</font>. It <font color='blue'>connects</font> a <font color='blue'>model</font> with its necessary <font color='blue'>preprocessing</font> and <font color='blue'>postprocessing steps</font>, allowing us to directly input any text and get an intelligible answer:

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
classifier("I've been waiting for a HuggingFace course my whole life.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

We can even pass <font color='blue'>several sentences</font>!

In [4]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.",
     "I hate this so much!",
     "I like to eat chocolate and peanut butter.",
     "Do you like green eggs and ham?",
     "What is better than running?",
     "I don't like walking without socks."]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455},
 {'label': 'POSITIVE', 'score': 0.9995144605636597},
 {'label': 'POSITIVE', 'score': 0.8833526968955994},
 {'label': 'NEGATIVE', 'score': 0.9764426350593567},
 {'label': 'POSITIVE', 'score': 0.8040932416915894}]

By default, this pipeline <font color='blue'>selects</font> a <font color='blue'>particular pretrained model</font> that has been <font color='blue'>fine-tuned</font> for <font color='blue'>sentiment analysis</font> in English. The model is <font color='blue'>downloaded</font> and <font color='blue'>cached</font> when you create the <font color='blue'>classifier</font> object. If you <font color='blue'>rerun</font> the <font color='blue'>command</font>, the <font color='blue'>cached model</font> will be <font color='blue'>used</font> instead and there is no need to download the model again.

There are <font color='blue'>three main steps</font> involved when you pass some text to a pipeline:

1. The text is <font color='blue'>preprocessed</font> into a format the model can understand.
2. The <font color='blue'>preprocessed inputs</font> are <font color='blue'>passed</font> to the <font color='blue'>model</font>.
3. The <font color='blue'>predictions</font> of the model are <font color='blue'>post-processed</font>, so you can make sense of them.

Some of the currently [available pipelines](https://huggingface.co/transformers/main_classes/pipelines) are:

1. feature-extraction (get the vector representation of a text)
2. fill-mask
3. ner (named entity recognition)
4. question-answering
5. sentiment-analysis
6. summarization
7. text-generation
8. translation
9. zero-shot-classification

Let's have a look at a few of these!

### Zero-shot classification

We'll start by tackling a <font color='blue'>more challenging</font> task where we need to <font color='blue'>classify texts</font> that <font color='blue'>haven't been labelled</font>. This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise. For this use case, the <font color='blue'>zero-shot-classification</font> pipeline is very powerful: it allows you to specify <font color='blue'>which labels to use for the classification</font>, so you <font color='blue'>don't have to rely on the labels of the pretrained model</font>. You've already seen how the model can classify a sentence as positive or negative using those two labels — but it can also classify the text using any other set of labels you like.

In [8]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445968627929688, 0.1119757816195488, 0.04342734441161156]}

In [9]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier(
    "My favoite sport involves running shoes and long distances.",
    candidate_labels=["health", "education", "politics"],
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'My favoite sport involves running shoes and long distances.',
 'labels': ['health', 'education', 'politics'],
 'scores': [0.7021063566207886, 0.18310600519180298, 0.11478759348392487]}

This <font color='blue'>pipeline</font> is called <font color='blue'>zero-shot</font> because you <font color='blue'>don't need</font> to <font color='blue'>fine-tune the model</font> on your data to use it. It can directly return probability scores for any list of labels you want!

✏️ **Try it out!** <font color='blue'>Play</font> around with your own <font color='blue'>sequences</font> and <font color='blue'>labels</font> and see how the model behaves.

In [10]:
# Exercise
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier(
    "The best running shoes hold up for at least 600 miles.",
    candidate_labels=["Brooks", "Adidas", "Kona"],
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'The best running shoes hold up for at least 600 miles.',
 'labels': ['Brooks', 'Kona', 'Adidas'],
 'scores': [0.39663437008857727, 0.31448471546173096, 0.2888808846473694]}

### Text generation

Now let's see how to <font color='blue'>use</font> a <font color='blue'>pipeline</font> to <font color='blue'>generate some text</font>. The main idea here is that you <font color='blue'>provide a prompt</font> and the <font color='blue'>model will auto-complete</font> it by <font color='blue'>generating</font> the remaining <font color='blue'>text</font>. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it's normal if you don't get the same results as shown below.

In [6]:
from transformers import pipeline

generator = pipeline("text-generation", model="openai-community/gpt2")
generator("In this course, we will teach you how to",
          max_length=20, # Length of sentence
          num_return_sequences=4) # 4 sentences

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In this course, we will teach you how to implement the Racket program into the Raspberry Pi's"},
 {'generated_text': 'In this course, we will teach you how to create a virtual machine like Linux, which uses the'},
 {'generated_text': 'In this course, we will teach you how to read and write a complex code, which I shall'},
 {'generated_text': 'In this course, we will teach you how to use both of them in order to get rid of'}]

✏️ **Try it out!** Use the <font color='blue'>num_return_sequences</font> and <font color='blue'>max_length</font> arguments to generate <font color='blue'>two sentences</font> of <font color='blue'>15 words</font> each.

In [12]:
# Exercise
generator = pipeline("text-generation", model="distilbert/distilgpt2")
generator("We can explain machine learning by",
          max_length=15, # Length of sentence
          num_return_sequences=2) # 2 sentences

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'We can explain machine learning by seeing the same thing:\n\n\n\n'},
 {'generated_text': 'We can explain machine learning by studying neural networks during a task, but it'}]

### Using any model from the Hub in a pipeline

The previous examples used the <font color='blue'>default model</font> for the task at hand, but you can also <font color='blue'>choose</font> a <font color='blue'>particular model</font> from the <font color='blue'>Hub</font> to use in a pipeline for a specific task — say, text generation. Go to the [Model Hub](https://huggingface.co/models) and <font color='blue'>click</font> on the <font color='blue'>corresponding tag</font> on the <font color='blue'>left</font> to display only the supported models for that task. You should get to a page like [this one](https://huggingface.co/models?pipeline_tag=text-generation).

Let's try the [distilgpt2](https://huggingface.co/distilgpt2) model! Here's how to load it in the same pipeline as before:

In [14]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to set your course:\n\n\n\n\nCreate and set the new variable to use to manage the'},
 {'generated_text': 'In this course, we will teach you how to develop a unique brand of custom-built machine parts to the user experience, from to the user experience'}]

And similarly for the <font color='blue'>gpt2-large</font> model:

In [16]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2-large")

generator(
    "Boys like to do the following activities in the summer",
    max_length=30,
    num_return_sequences=5,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Boys like to do the following activities in the summer holidays;\n\n• to get up, to cook, to play football (with a friend'},
 {'generated_text': 'Boys like to do the following activities in the summer of every year (all activities are done as often but as little, if at all, as'},
 {'generated_text': 'Boys like to do the following activities in the summer:\n\n- Take a break from summer vacation time to go to the beach, where there'},
 {'generated_text': "Boys like to do the following activities in the summer when it's hot:\n\nVisit beach playgrounds\n\nVisit the lake\n\nPlay"},
 {'generated_text': 'Boys like to do the following activities in the summer – for reasons that are totally self-evident. (If the weather is too warm to'}]

You can <font color='blue'>refine</font> your <font color='blue'>search</font> for a model by <font color='blue'>clicking</font> on the <font color='blue'>language tags</font>, and pick a model that will generate text in another language. The <font color='blue'>Model Hub</font> even contains <font color='blue'>checkpoints</font> for <font color='blue'>multilingual models</font> that support several languages.

Once you select a model by clicking on it, you'll see that there is a <font color='blue'>widget</font> enabling you to <font color='blue'>try it directly online</font>. This way you can quickly test the model's capabilities before downloading it.

✏️ **Try it out!** Use the filters to find a text generation model for another language. Feel free to play with the widget and use it in a pipeline!

I will try text generation for the <font color='blue'>Japanese language</font>:

In [ ]:
# Exercise
from transformers import pipeline

generator = pipeline("text-generation", model="ai-forever/mGPT")

japanese_text = "ティーンエイジャーは冬にこれらのスポーツを楽しむことが好きです。"

generated_text = generator(
    japanese_text,
    max_length=30,
    num_return_sequences=1,
)

The generated text (in Japanese) is:

In [13]:
generated_text

[{'generated_text': 'ティーンエイジャーは冬にこれらのスポーツを楽しむことが好きです。 私は、私の息子が、私の母親が、'}]

### The Inference API

All the models can be <font color='blue'>tested</font> directly through your <font color='blue'>browser</font> using the <font color='blue'>Inference API</font>, which is available on the Hugging Face [website](https://huggingface.co/). You can play with the model directly on this page by inputting custom text and watching the model process the input data.

The Inference API that powers the widget is <font color='blue'>also available </font>as a <font color='blue'>paid product</font>, which comes in handy if you need it for your workflows. See the [pricing page](https://huggingface.co/pricing) for more details.

### Mask filling

The next pipeline you'll try is <font color='blue'>fill-mask</font>. The idea of this task is to <font color='blue'>fill</font> in the <font color='blue'>blanks</font> in a given text:

In [4]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="distilbert-base-uncased")
unmasker("This course will teach you all about [MASK] models.", top_k=2)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.09235832095146179,
  'token': 8045,
  'token_str': 'mathematical',
  'sequence': 'this course will teach you all about mathematical models.'},
 {'score': 0.027659853920340538,
  'token': 2535,
  'token_str': 'role',
  'sequence': 'this course will teach you all about role models.'}]

✏️ **Try it out!** Search for the <font color='blue'>bert-base-cased</font> model on the <font color='blue'>Hub</font> and <font color='blue'>identify</font> its <font color='blue'>mask word</font> in the <font color='blue'>Inference API widget</font>. What does this model predict for the sentence in our pipeline example above?

In [6]:
# Exercise
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-cased")
unmasker("This course will teach you all about [MASK] models.", top_k=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.2596302330493927,
  'token': 1648,
  'token_str': 'role',
  'sequence': 'This course will teach you all about role models.'},
 {'score': 0.09427274763584137,
  'token': 1103,
  'token_str': 'the',
  'sequence': 'This course will teach you all about the models.'}]

The <font color='blue'>top_k argument</font> controls how many <font color='blue'>possibilities</font> you want to be displayed. Note that here the <font color='blue'>model</font> fills in the special [MASK} word, which is often referred to as a <font color='blue'>mask token</font>. <font color='blue'>Other mask-filling models</font> might have <font color='blue'>different mask tokens</font>, so it's always good to verify the proper mask word when exploring other models. One way to check it is by looking at the mask word used in the widget.

### Named entity recognition

Named entity recognition (<font color='blue'>NER</font>) is a task where the model has to <font color='blue'>find</font> which <font color='blue'>parts</font> of the <font color='blue'>input text correspond</font> to entities such as <font color='blue'>persons</font>, <font color='blue'>locations</font>, or <font color='blue'>organizations</font>. Let's look at an example:

In [8]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True, model="dbmdz/bert-large-cased-finetuned-conll03-english")
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

Let's also examine the same example with the <font color='blue'>RoBERTa</font> model:

In [10]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True, model="xlm-roberta-large-finetuned-conll03-english")
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER',
  'score': 0.9998966,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9998043,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99908185,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

Here the model <font color='blue'>correctly identified</font> that <font color='blue'>Sylvain</font> is a <font color='blue'>person</font> (PER), <font color='blue'>Hugging Face</font> an <font color='blue'>organization</font> (ORG), and <font color='blue'>Brooklyn</font> a <font color='blue'>location</font> (LOC).

We pass the option <font color='blue'>grouped_entities=True</font> in the pipeline creation function to tell the pipeline to <font color='blue'>regroup together</font> the <font color='blue'>parts</font> of the sentence that correspond to the <font color='blue'>same entity</font>: here the model correctly grouped “Hugging” and “Face” as a single organization, even though the name consists of multiple words. In fact, as we will see in the next chapter, the <font color='blue'>preprocessing</font> even <font color='blue'>splits</font> some <font color='blue'>words</font> into <font color='blue'>smaller parts</font>. For instance, Sylvain is split into four pieces: S, ##yl, ##va, and ##in. In the post-processing step, the pipeline successfully regrouped those pieces.

✏️ **Try it out!** <font color='blue'>Search</font> the <font color='blue'>Model Hub</font> for a model able to do <font color='blue'>part-of-speech tagging</font> (usually abbreviated as POS) in English. What does this model predict for the sentence in the example above?

In [12]:
ner = pipeline("ner", grouped_entities=True, model="QCRI/bert-base-multilingual-cased-pos-english")
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PRP$',
  'score': 0.99944586,
  'word': 'My',
  'start': 0,
  'end': 2},
 {'entity_group': 'NN',
  'score': 0.9995615,
  'word': 'name',
  'start': 3,
  'end': 7},
 {'entity_group': 'VBZ',
  'score': 0.9995523,
  'word': 'is',
  'start': 8,
  'end': 10},
 {'entity_group': 'NNP',
  'score': 0.9963742,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'CC',
  'score': 0.9996537,
  'word': 'and',
  'start': 19,
  'end': 22},
 {'entity_group': 'PRP',
  'score': 0.99956113,
  'word': 'I',
  'start': 23,
  'end': 24},
 {'entity_group': 'VBP',
  'score': 0.9976095,
  'word': 'work',
  'start': 25,
  'end': 29},
 {'entity_group': 'IN',
  'score': 0.999801,
  'word': 'at',
  'start': 30,
  'end': 32},
 {'entity_group': 'NNP',
  'score': 0.96980053,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'IN',
  'score': 0.99977857,
  'word': 'in',
  'start': 46,
  'end': 48},
 {'entity_group': 'NNP',
  'score': 0.99542207,
  'word': 'Brookly

### Question answering

The <font color='blue'>question-answering</font> pipeline <font color='blue'>answers questions</font> using information from a given context.

Let's firs try this with <font color='blue'>DistilBERT</font>:

In [14]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.6949753165245056, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

Then with <font color='blue'>BERT large model (uncased)</font>:

In [16]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.5150135159492493,
 'start': 33,
 'end': 57,
 'answer': 'Hugging Face in Brooklyn'}

Note that this pipeline works by <font color='blue'>extracting information</font> from the <font color='blue'>provided context</font>; it does not generate the answer.

### Summarization

<font color='blue'>Summarization</font> is the task of <font color='blue'>reducing</font> a <font color='blue'>text</font> into a shorter text <font color='blue'>while keeping</font> all (or <font color='blue'>most</font>) of the <font color='blue'>important aspects</font> referenced in the text. Here's an example:

In [18]:
from transformers import pipeline

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", max_length=60)
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': ' The number of engineering graduates in the United States has declined in recent years . China and India graduate six and eight times as many traditional engineers as the U.S. does . Rapidly developing economies such as India and Europe continue to encourage and advance the teaching of engineering .'}]

Like with <font color='blue'>text generation</font>, you can specify a <font color='blue'>max_length</font> or a <font color='blue'>min_length</font> for the <font color='blue'>esult</font>.

### Translation

For translation, you can use a <font color='blue'>default model</font> if you <font color='blue'>provide</font> a <font color='blue'>language pair</font> in the <font color='blue'>task name</font> (such as "translation_en_to_fr"), but the easiest way is to pick the model you want to use on the [Model Hub](https://huggingface.co/models). Here we'll try translating from <font color='blue'>French to English</font>:

In [20]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'This course is produced by Hugging Face.'}]

Like with <font color='blue'>text generation</font> and <font color='blue'>summarization</font>, you can specify a <font color='blue'>max_length</font> or a <font color='blue'>min_length</font> for the <font color='blue'>result</font>.

The <font color='blue'>pipelines</font> shown so far are mostly for <font color='blue'>demonstrative purposes</font>. They were programmed for <font color='blue'>specific tasks</font> and <font color='blue'>cannot perform variations</font> of them. In the next chapter, you'll learn what's inside a `pipeline()` function and how to customize its behavior.

✏️ **Try it out!** Search for translation models in other languages and try to translate the previous sentence into a few different languages.

First, we'll try translating from <font color='blue'>French to Spanish</font>:

In [22]:
# French to Spanish
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-es")
translator("Ce cours est produit par Hugging Face.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Este curso es producido por Hugging Face.'}]

Then from <font color='blue'>French to German</font>:

In [24]:
# French to German
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-de")
translator("Ce cours est produit par Hugging Face.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Dieser Kurs wird von Hugging Face erstellt.'}]